In [1]:
import pandas as pd

In [2]:
INPUT_DATA_FOLDER = "../data"
REPORTS_DATA_FILE = "ISW_vector.csv"

OUTPUT_FOLDER = "../data/all_data_preprocessed"
ISW_OUTPUT_DATA_FILE = "all_isw.csv"
ALARMS_OUTPUT_DATA_FILE = "all_alarms.csv"
WEATHER_EVENTS_OUTPUT_DATA_FILE = "all_weather_by_hour.csv"
MERGED_OUTPUT_DATA_FILE = "weather_regions_alarms_merged.csv"
ALL_MERGED_DATA_FILE = "all_merged"

MODEL_FOLDER = "model"

tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v1"
count_vectorizer_version = "v1"

In [3]:
df = pd.read_parquet(f"{OUTPUT_FOLDER}/{ALL_MERGED_DATA_FILE}.parquet")

In [5]:
df.columns


Index(['day_datetimeEpoch', 'day_tempmax', 'day_tempmin', 'day_temp',
       'day_dew', 'day_humidity', 'day_precip', 'day_precipcover',
       'day_solarradiation', 'day_solarenergy', 'day_uvindex', 'day_sunrise',
       'day_sunset', 'day_moonphase', 'hour_datetime', 'hour_datetimeEpoch',
       'hour_temp', 'hour_humidity', 'hour_dew', 'hour_precip',
       'hour_precipprob', 'hour_snow', 'hour_snowdepth', 'hour_preciptype',
       'hour_windgust', 'hour_windspeed', 'hour_winddir', 'hour_pressure',
       'hour_visibility', 'hour_cloudcover', 'hour_uvindex', 'hour_conditions',
       'center_city_ua', 'center_city_en', 'region_alt', 'region_id',
       'event_region_city', 'event_all_region', 'event_start', 'event_end',
       'event_start_hour', 'event_end_hour', 'event_start_hour_datetimeEpoch',
       'event_end_hour_datetimeEpoch', 'event_hour_level_event_time',
       'event_hour_level_event_datetimeEpoch', 'keywords', 'report_date',
       'date_tomorrow_datetime', 'alarms_in_

In [8]:
df[['hour_datetimeEpoch','alarms_in_regions']].head(6)

,hour_datetimeEpoch,alarms_in_regions
0,1645653600,0.0
1,1645657200,0.0
2,1645660800,0.0
3,1645664400,0.0
4,1645668000,0.0
5,1645671600,0.0
